In [1]:
import pandas as pd
import numpy as np
from plotnine import *
import matplotlib.pyplot as plt

In [16]:
df = pd.read_parquet("multi_model_1000_0.9_10000_100_RMSE_0.95_600_1/results.parquet")

In [3]:
df

,hyper_idx,h,observed,prediction,lower_bound,upper_bound,coverage_flags,squared_error,pred_length,flops,num_params,sd,runtime
0,0,1,0.145252,-0.235941,-4.530580,4.058700,True,0.145308,8.589279,59.24,12.20,3.058905,3.257592
1,0,2,-0.751520,-0.163570,-4.201818,3.874680,True,0.345684,8.076498,59.24,12.20,3.058905,3.257592
2,0,3,-0.225278,-0.063578,-3.450679,3.323525,True,0.026147,6.774204,59.24,12.20,3.058905,3.257592
3,0,4,0.464652,-0.213394,-4.681186,4.254401,True,0.459747,8.935587,59.24,12.20,3.058905,3.257592
4,0,5,-0.393433,-0.216828,-4.580473,4.146821,True,0.031189,8.727294,59.24,12.20,3.058905,3.257592
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,999,96,-1.616958,-0.264844,-6.765475,6.235790,True,1.828213,13.001265,357.30,111.65,3.058905,1.316972
96,999,97,-1.524002,-0.088345,-6.404932,6.228245,True,2.061111,12.633177,357.30,111.65,3.058905,1.316972
97,999,98,-1.415987,-0.164516,-5.500917,5.171887,True,1.566177,10.672805,357.30,111.65,3.058905,1.316972
98,999,99,-0.482017,-0.303352,-6.561142,5.954443,True,0.031921,12.515585,357.30,111.65,3.058905,1.316972


In [18]:
results = df.groupby(["hyper_idx", "h"]).agg(
        mean_squared_error=("squared_error", "mean"),
        RMSE=("squared_error", lambda x: np.sqrt(x.mean())),
        MAD=("squared_error", lambda x: np.median(np.sqrt(x))),
        coverage_accuracy=("coverage_flags", "mean"),
        sd = ("sd", "mean")
    )

results["normalized_rmse"] = results["RMSE"] / results["sd"]

results.drop(columns="sd", inplace=True)

In [19]:
results

mean_squared_error      RMSE       MAD  coverage_accuracy  \
hyper_idx h                                                                
0         1              9.370152  3.061070  2.032591               0.98   
          2              9.635329  3.104083  1.940782               0.96   
          3              9.891620  3.145095  2.108933               0.93   
          4              9.217954  3.036108  1.949531               0.98   
          5             10.295859  3.208716  2.191584               0.99   
...                           ...       ...       ...                ...   
999       96            10.237539  3.199615  2.177126               0.96   
          97            10.355869  3.218054  1.963572               0.97   
          98            10.434323  3.230220  2.131124               0.93   
          99            10.950534  3.309159  2.289940               0.95   
          100           11.586057  3.403830  2.389713               0.95   

               normalized_rmse  
hyper_idx h                     
0         1           1.000708  
          2           1.014769  
          3           1.028177  
          4           0.992547  
          5           1.048975  
...                        ...  
999       96          1.046000  
          97          1.052028  
          98          1.056005  
          99          1.081812  
          100         1.112761  

[100000 rows x 5 columns]

In [20]:
results.groupby(level="hyper_idx").diff()

mean_squared_error      RMSE       MAD  coverage_accuracy  \
hyper_idx h                                                                
0         1                   NaN       NaN       NaN                NaN   
          2              0.265177  0.043012 -0.091809              -0.02   
          3              0.256291  0.041012  0.168151              -0.03   
          4             -0.673666 -0.108986 -0.159402               0.05   
          5              1.077905  0.172608  0.242053               0.01   
...                           ...       ...       ...                ...   
999       96            -0.400496 -0.061985 -0.068768              -0.01   
          97             0.118330  0.018438 -0.213555               0.01   
          98             0.078454  0.012167  0.167552              -0.04   
          99             0.516211  0.078939  0.158815               0.02   
          100            0.635523  0.094671  0.099773               0.00   

               normalized_rmse  
hyper_idx h                     
0         1                NaN  
          2           0.014061  
          3           0.013407  
          4          -0.035629  
          5           0.056428  
...                        ...  
999       96         -0.020264  
          97          0.006028  
          98          0.003977  
          99          0.025806  
          100         0.030949  

[100000 rows x 5 columns]